In [2]:
import numpy as np
import os
import threading
import operator
import pickle
import re
import pandas as pd


start_time = 1190146243
end_time = 1192994591

In [3]:
def get_lemma_distribution(file_name):
    lemmas=[]
    with open(file_name) as f:
        for line in f:
            url=line.strip().split(' ')[2]
            if url[7:9] != 'en':
                continue
            lemma=url[url.rfind('/')+1:]
            if ':' in lemma:
                continue
            lemmas.append(lemma)
    return lemmas


def count_lemma(dictionary, lemmas):
    for lemma in lemmas:
        if lemma not in dictionary:
            dictionary[lemma] = 1
        else:
            dictionary[lemma] += 1

            
def get_lemma_counter(num, nworkers = 4):
    data_root = "./data/only_lemma/"
    data_files = os.listdir(data_root)
    counter = {}

    threads = [None] * nworkers
    flags = [True] * nworkers
    
    if num > len(data_files):
        num = len(data_files)
    indices = np.random.choice(len(data_files), num)
    
    def target(index, flags, counter, filename):
        count_lemma(counter, get_lemma_distribution(filename))
        flags[index] = True
        
    j = 1
    
    for i in indices:
        index = 0
        while True:
            try:
                index = flags.index(True)
                flags[index] = False
                break
            except:
                continue
        print("Processing file {}".format(j))
        j += 1
        t = threading.Thread(target = target, args = [index, flags, counter, data_root + data_files[i]])
        threads[index] = t
        t.start()
        
    for thread in threads:
        if thread:
            thread.join()
    return counter

def get_url(entry):
    url = "https://en.wikipedia.org/wiki/{}".format(entry)
    return url
    
def sort_by_value(dictionary):
    sorted_dict = sorted(sort_by_value.items(), key=operator.itemgetter(1))
    return sorted_dict

def dump_data(data, output_name):
    with open(output_name, 'wb') as f:
        pickle.dump(data, f)

def load_data(input_name):
    with open(input_name, 'rb') as f:
        data = pickle.load(f)
    return data

def dump_word_frequency(data, output_dir, num = 2000):
    sorted_dict = sorted(counter.items(), key=operator.itemgetter(1), reverse=True)
    with open(output_dir+'hot_{}.csv'.format(num), 'w') as f:
        f.write("entry\tfrequency\n")
        for i in np.random.choice(100000, num):
            item = sorted_dict[i]
            if len(item[0]) == 0:
                continue
            f.write("{}\t{}\n".format(item[0], item[1]))

In [5]:
word_frequency = pd.Series.from_csv('data/revisions/hot_100000.csv',sep='\t')

/home/xuanli2/penv/lib/python3.5/site-packages/pandas/core/series.py:2890: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


In [6]:
word_frequency

Main_Page                                        13601719
Wiki                                               444697
Halo_3                                             217926
Wikipedia                                          148431
Naruto                                             120657
United_States                                      107160
Deaths_in_2007                                     102920
Sex                                                 97584
Heroes_%28TV_series%29                              94180
HTTP                                                88070
Love                                                87235
MySpace                                             87024
YouTube                                             81909
sort_none.gif                                       77914
sort_down.gif                                       77278
sort_up.gif                                         77181
Politics                                            75759
Cocaine       